In [1]:
from itertools import product
from keras.layers import Input, Activation, BatchNormalization, Dense, Conv2D, concatenate, Reshape, MaxPooling2D
from keras import Model, Sequential
from keras.activations import relu, sigmoid
from keras.losses import logcosh, mse, mae, mape
import pandas as pd
import numpy as np
import os
from keras.models import load_model, model_from_json
import json
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import joblib
import h5py
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import keras
from matplotlib import pyplot as plt
from collections import OrderedDict
import collections


def save_model(model_save_name, model):
    with open(model_save_name + '.json', 'w') as json_file:
        json_file.write(model.to_json())

    model.save_weights(model_save_name + '.h5')
    
    
def load_model_files(model_name):
    json_file = open(model_name + '.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model = model_from_json(loaded_model_json)
    model.load_weights(model_name + '.h5')
    
    return model


class MuonML:
    def __init__(self, params, experiment_name=None, X_train=None, Y_train=None, X_test=None, Y_test=None):
        """ Constructor """
        
        self.name = experiment_name
        
        self.X_train = X_train
        self.Y_train = Y_train
        self.X_test = X_test
        self.Y_test = Y_test
        self.X_scaler = None
        self.Y_scaler = None
        
        self.params = params
        
        self.models = []
        self.histories = []
        self.results = []
        self.models_info = None
        
        print('Creating experiments...')
        self.experiments = [dict(zip(self.params.keys(), value)) for value in product(*self.params.values())]
        print(len(self.experiments))
        
        self.__create_list_of_models()
        self.__create_model_info()
        if X_train != None:
            self.__create_folder_structure(self.name)
        
        print('Created ' + str(len(self.experiments)) + ' models')
    
    
    def __create_folder_structure(self, name):
        """ Creates a folder structure to save info about models """
        
        if not os.path.exists(name):
            os.makedirs(name)
            
        for i, model in enumerate(self.models):
            folder_name = name + '/' + 'model_' + str(i)
            if not os.path.exists(folder_name):
                os.makedirs(folder_name)
        
        
    def __create_model_info(self):
        """ Creates a dataframe which provides description of all different models """
        
        data = []
        for i, exp in enumerate(self.experiments):
            str_values = []
            for value in exp.values():
                str_values.append(str(value))
            data.append(str_values)
        data = np.array(data)
        
        self.models_info = pd.DataFrame(data=data, columns=self.params.keys())
        self.models_info.index.name = 'model_id'
    
    
    def __create_list_of_models(self):
        """ Creates a grid from params and makes a list of models """
        
        self.models = [] # list of keras models
        
        for exp in self.experiments:
            print(self.experiments.index(exp))
            k_reg = exp['kernel_regularizer'](exp['regularizer_factor'])
            a_reg = exp['activity_regularizer'](exp['regularizer_factor'])

            inputA = Input(shape = (38, 20, 1))
            inputB = Input(shape = (38, 20, 1))

            #x0 = BatchNormalization(momentum=0.9)(inputA)
            x0 = Conv2D(exp['conv_first_nodes'], (3,3))(inputA)
            x0 = Activation('relu')(x0)
            if exp['bn_0']:
                x0 = BatchNormalization()(x0)

            x0 = Conv2D(exp['conv_second_nodes'], (3,3))(x0)
            x0 = Activation('relu')(x0)
            if exp['bn_1']:
                x0 = BatchNormalization()(x0)
            
            x0 = Conv2D(exp['conv_third_nodes'], (3,3))(x0)
            x0 = Activation('relu')(x0)
            if exp['bn_2']:
                x0 = BatchNormalization()(x0)

            x0 = MaxPooling2D((2,2), strides=(2,2))(x0)

            #x1 = BatchNormalization(momentum=0.9)(inputB)
            x1 = Conv2D(exp['conv_first_nodes'], (3,3))(inputB)
            x1 = Activation('relu')(x1)
            if exp['bn_0']:
                x1 = BatchNormalization()(x1)

            x1 = Conv2D(exp['conv_second_nodes'], (3,3))(x1)
            x1 = Activation('relu')(x1)
            if exp['bn_1']:
                x1 = BatchNormalization()(x1)

            x1 = Conv2D(exp['conv_third_nodes'], (3,3))(x1)
            x1 = Activation('relu')(x1)
            if exp['bn_2']:
                x1 = BatchNormalization()(x1)

            x1 = MaxPooling2D((2,2), strides=(2,2))(x1)

            x = concatenate([x0, x1])

            target_shape = x.shape[1] * x.shape[2] * x.shape[3]
            x = Reshape(target_shape=(int(target_shape),))(x)

            x = Dense(256, kernel_regularizer=k_reg, activity_regularizer=a_reg)(x)
            x = Activation('relu')(x)
            if exp['bn_3']:
                x = BatchNormalization()(x)

            x = Dense(256, kernel_regularizer=k_reg, activity_regularizer=a_reg)(x)
            x = Activation('relu')(x)
            if exp['bn_4']:
                x = BatchNormalization()(x)

            x = Dense(1, activation='linear')(x)

            model = Model([inputA, inputB], x)
            model.compile(loss=exp['loss'], optimizer=exp['optimizer']())
            
            self.models.append(model)
            
    
    def train_models(self):
        """ Train all models and save json, h5 and history """
        
        self.histories = []
        for i, exp in enumerate(self.experiments):
            print('Training model ' + str(i + 1) + '/' + str(len(self.experiments)))
            model = self.models[i]
            history = model.fit(self.X_train, self.Y_train, epochs=exp['epochs'],\
                        batch_size=exp['batch_size'], verbose=1, validation_split=exp['validation_split'])
            self.histories.append(history)
            
            # Save model info
            model_path = self.name + '/' + 'model_' + str(i) + '/model_' + str(i)
            save_model(model_path, model)
            
            # Save model history
            with open(model_path + '_history.json', 'w') as f:
                json.dump(str(history.history), f)
            
    
    def evaluate_models(self, metrics):
        """ Evaluate all models and save results """
        
        self.results = []
        for i, exp in enumerate(self.experiments):
            print('Evaluating model ' + str(i + 1) + '/' + \
                  str(len(self.experiments)) + ' for metrics ' + str(metrics))
            model = self.models[i]
            model.compile(loss=exp['loss'], optimizer=exp['optimizer'](), metrics=metrics)
            result = model.evaluate(self.X_test, self.Y_test, batch_size=2048)
            
            self.results.append(result)
            
            # Save result to file
            model_path = self.name + '/' + 'model_' + str(i) + '/model_' + str(i)
            with open(model_path + '_evaluation.csv', 'w') as f:
                f.write(str(result)[1:-1])
            
        results_df = pd.DataFrame(data=self.results, columns=self.models[0].metrics_names)
        self.models_info = self.models_info.join(results_df, on='model_id', how='outer', lsuffix='_functions')
        self.models_info.to_csv(self.name + '/results_overview.csv')
        
        
    def predict(self):
        """ Make predictions from models and save results """
        
        for i, exp in enumerate(self.experiments):
            print('Predicting outputs of the current test set, model ' + \
                  str(i + 1) + '/' + str(len(self.experiments)))
            
            model = self.models[i]
            model.compile(loss=exp['loss'], optimizer=exp['optimizer']())
            
            result = model.predict(self.X_test, batch_size=256)
            result = np.array(np.squeeze(result)).T
            
            model_path = self.name + '/' + 'model_' + str(i) + '/model_' + str(i)
            
            if self.Y_scaler == None:
                np.savetxt(model_path + '_predictions.csv', result, delimiter=',')
            else:
                np.savetxt(model_path + '_predictions_scaled.csv', result, delimiter=',')
                results_rescaled = self.Y_scaler.inverse_transform(result)
                np.savetxt(model_path + '_predictions.csv', results_rescaled, delimiter=',')
                
        
    def perform_CV(self, X, Y, metrics, n_fold=5, X_scaler=None, Y_scaler=None, \
                   save_predictions=False, stop_after=None):
        """ Run nfold cross validation """
        
        base_name = self.name
        test_set_length = int(len(X[0]) / n_fold)
        
        cv_info_dfs = []
        
        for i in range(n_fold):
            if stop_after != None and i >= stop_after:
                break
            
            print('Performing ' + str(i + 1) + '/' + str(n_fold) + ' iteration of cross validation.')
            
            self.name = base_name + '_CV/CV_' + str(i)
            
            test_start = i * test_set_length
            test_end = i * test_set_length + test_set_length
            
            # Create train/test sets for current fold of CV
            X = np.array(X)
            
            self.X_train = np.concatenate((X[:, 0:test_start, :, :], X[:, test_end:, :, :]), axis=1)
            self.Y_train = np.concatenate((Y[0:test_start], Y[test_end:]), axis=0)
            self.X_test = X[:, test_start:test_end, :, :]
            self.Y_test = Y[test_start:test_end]
            
            print(self.X_train.shape)
            print(self.Y_train.shape)
            
            print(self.X_test.shape)
            print(self.Y_test.shape)
            
            
            self.X_train = list(self.X_train)
            self.X_test = list(self.X_test)
            
            print(len(self.X_train))
            print(len(self.X_test))
            
            # Scale data
            if X_scaler != None and Y_scaler != None:
                self.X_scaler = X_scaler
                self.Y_scaler = Y_scaler
            
                self.X_train = X_scaler.fit_transform(self.X_train)
                self.Y_train = Y_scaler.fit_transform(self.Y_train)

                self.X_test = X_scaler.transform(self.X_test)
                self.Y_test = Y_scaler.transform(self.Y_test)
            
            # Setup models for current fold
            self.__create_list_of_models()
            self.__create_model_info()
            self.__create_folder_structure(self.name)
            self.train_models()
            self.evaluate_models(metrics)
            
            if save_predictions:
                self.predict()
            
            # Save scalers
            joblib.dump(self.X_scaler, self.name + '/X_scaler.dat')
            joblib.dump(self.Y_scaler, self.name + '/Y_scaler.dat')
            
            cv_info_dfs.append(self.models_info)
        
        # Save average results by model in a dataframe
        columns_to_group_by = ['model_id']
        for param in self.params.keys():
            columns_to_group_by.append(param)
        
        average_df = pd.concat(cv_info_dfs).groupby(by=columns_to_group_by).mean()
        self.models_info = average_df
        self.name = base_name + '_CV'
        average_df.to_csv(self.name + '/results_overview.csv')
    
    
    def get_model(self, model_id):
        return self.models[model_id]
    
    
    def get_model_history(self, model_id):
        return self.histories[model_id]
    
    
    def get_model_evaluation(self, model_id):
        return self.results[model_id]
    
    
    def get_best_model_id(self, metrics, idmax=False):
        if idmax:
            return self.models_info[metrics].idxmax()
        else:
            return self.models_info[metrics].idxmin()
        

    def load_models_info(self, name):
        """ Load model's info from a folder. If loading from CV, load from a specific fold, not everything """
        """ Load: all models h5 files, all models evaluation, all models history, total evaluation """
        
        self.name = name
        
        self.models_info = pd.read_csv(self.name + '/results_overview.csv')
        
        model_dirs = [x[0] for x in os.walk(self.name)]
        model_dirs = model_dirs[1:]
        print(model_dirs)
        
        for i, model_dir in enumerate(model_dirs):
            model_path = model_dir + '/model_' + str(i)
            
            if '.ipynb_checkpoints' in model_path:
                continue
            
            model = load_model_files(model_path)
            self.models.append(model)
            
            with open(model_path + '_history.json', 'r') as json_file:
                history = json.load(json_file)
            self.histories.append(history)
            
            with open(model_path + '_evaluation.csv', 'r') as f:
                result = f.readlines()
            self.results.append(list(result))
            
            
    def plot_history_info(self, key):
        pass
        

Using TensorFlow backend.


In [2]:
def print_stats(lst, name='', rnd=5):
    print(name)
    print('mean = ' + str(round(np.mean(lst), rnd)))
    print('std  = ' + str(round(np.std(lst), rnd)))
    print('min  = ' + str(round(np.min(lst), rnd)))
    print('max  = ' + str(round(np.max(lst), rnd)) + '\n')

Reading data

In [ ]:
def get_matched(X, Y, Y_cl3d):
    map_ids_Y = {x[0]: i for i, x in enumerate(Y) if np.any(X[i,:36,:20,:] > 0)} # Set of ids in Y, if E > 0 in X
    map_ids_Y_cl3d = {x[0]: i for i, x in enumerate(Y_cl3d)} # Set of ids in Y_cl3d

    # Match the data by ids

    X_matched = []
    Y_matched = []
    Y_cl3d_matched = []

    for id_Y in map_ids_Y:
        if id_Y in map_ids_Y_cl3d:
            ind_Y = map_ids_Y[id_Y]
            ind_Y_cl3d = map_ids_Y_cl3d[id_Y]

            X_matched.append(X[ind_Y, : , :20, :])
            Y_matched.append(Y[ind_Y, 4])

            Y_cl3d_matched.append(Y_cl3d[ind_Y_cl3d, 2])

    return X_matched, Y_matched, Y_cl3d_matched

files = [x for x in os.listdir('/home/dejan_notebooks/datasets/') if 'tc_dat_100' in x]

X_matched = []
Y_matched = []
Y_cl3d_matched = []

for filename in files:
    filename = '/home/dejan_notebooks/datasets/' + filename
    pileup_f = h5py.File(filename, 'r')
    X = pileup_f['Cl2d'][:]
    Y = pileup_f['Particle'][:] 
    Y_cl3d = pileup_f['Cl3d'][:]
    X = np.array(X)
    X = X.squeeze(1)
    Y = np.array(Y)
    Y_cl3d = np.array(Y_cl3d)
    
    X_m, Y_m, Y_cl3d_m = get_matched(X, Y, Y_cl3d)
    X_matched.extend(X_m)
    Y_matched.extend(Y_m)
    Y_cl3d_matched.extend(Y_cl3d_m)
    
    pileup_f.close()

X_matched = np.array(X_matched)
Y_matched = np.array(Y_matched)
Y_cl3d_matched = np.array(Y_cl3d_matched)

print(X_matched.shape)
print(Y_matched.shape)
print(Y_cl3d_matched.shape)



X_view_0 = X_matched.sum(axis=2)
X_view_1 = X_matched.sum(axis=3)

X_view_0_scaled = X_view_0 / 270
X_view_1_scaled = X_view_1 / 270

X_view_0 = np.expand_dims(X_view_0, -1)
X_view_1 = np.expand_dims(X_view_1, -1)

X_view_0_scaled = np.expand_dims(X_view_0_scaled, -1)
X_view_1_scaled = np.expand_dims(X_view_1_scaled, -1)

Y_matched_scaled = Y_matched / 1500
Y_cl3d_matched_scaled = Y_cl3d_matched / 1500

train_length = int(0.7 * len(Y_matched))

Y_train = Y_matched[:train_length]
Y_test = Y_matched[train_length:]

Y_cl3d_train = Y_cl3d_matched[:train_length]
Y_cl3d_test = Y_cl3d_matched[train_length:]

X_view_0_train = X_view_0[:train_length]
X_view_0_test = X_view_0[train_length:]

X_view_1_train = X_view_1[:train_length]
X_view_1_test = X_view_1[train_length:]

# Scaling

Y_train_scaled = Y_matched_scaled[:train_length]
Y_test_scaled = Y_matched_scaled[train_length:]

Y_cl3d_train_scaled = Y_cl3d_matched_scaled[:train_length]
Y_cl3d_test_scaled = Y_cl3d_matched_scaled[train_length:]

X_view_0_train_scaled = X_view_0_scaled[:train_length]
X_view_0_test_scaled = X_view_0_scaled[train_length:]

X_view_1_train_scaled = X_view_1_scaled[:train_length]
X_view_1_test_scaled = X_view_1_scaled[train_length:]

X_test_summed_E = np.apply_over_axes(np.sum, X_matched[train_length:], [1,2,3]).squeeze((1,2,3))

Preparing the model

In [ ]:
val_loss_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=20, cooldown=1, verbose=1, min_lr=0.0000001)
loss_lr = ReduceLROnPlateau(monitor='loss', factor=0.8, patience=20, cooldown=1, verbose=1, min_lr=0.0000001)

params = {
    'activation': [relu],
    'batch_size': [1024],
    'loss': [logcosh],
    'epochs': [300],
    'optimizer': [keras.optimizers.Adam],
    'validation_split': [0.2],
    'kernel_regularizer': [keras.regularizers.l2],
    'activity_regularizer': [keras.regularizers.l2],
    'regularizer_factor': [0.001, 0.0000001],
    'conv_first_nodes': [8, 16],
    'conv_second_nodes': [32, 64],
    'conv_third_nodes': [128, 256],
    'bn_0': [True, ],
    'bn_1': [True, ],
    'bn_2': [True, ],
    'bn_3': [True, ],
    'bn_4': [True, ],
    'callbacks': [[loss_lr, val_loss_lr]]
}

metrics=['mse']

In [ ]:
m = MuonML(params, 'hgcal_first_model')

In [ ]:
m.perform_CV([X_view_0, X_view_1], Y_matched, metrics, n_fold=5, X_scaler=None, Y_scaler=None, \
             save_predictions=True, stop_after=1)

In [ ]:
m.models_info

In [ ]:
for i, history in enumerate(m.histories):
    print(i)
    history = history.history
    m.models_info.iloc[i]
    #print(m.models_info.iloc[i]['losses'])
    plt.plot(history['loss'])
    plt.plot(history['val_loss'])
    #plt.ylim((0.0014, 0.0025))
    plt.show()